In [1]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tflearn


Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np

import tensorflow as tf
import tflearn
import random
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [5]:
with open("intents.json") as js_data:
    intents=json.load(js_data)
print("processing intents....")

processing intents....


In [6]:
words=[]
documents=[]
classes=[]
ignore_words=["!","?"]
print("looping through intent to convert them to words,classes,documents,ignorewords....")
for intent in intents["intents"]:
    for pattern in intent['patterns']:
        #tokenize every word in the sentence
        wi=nltk.word_tokenize(pattern)
        #add to our word list
        words.extend(wi)
        #add to documents in the corpus
        documents.append((wi,intent['tag']))
        # add to class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])              

looping through intent to convert them to words,classes,documents,ignorewords....


In [7]:
print('lowering stemming and removing duplicates..')
words=[stemmer.stem(wi.lower())for wi in words if wi not in ignore_words]
words=sorted(list(set(words)))

#remove duplicates
classes=sorted(list(set(classes)))
print(len(words),'unique stemmed words',words)
print(len(documents),"documents")
print(len(classes),"classes",classes)

lowering stemming and removing duplicates..
44 unique stemmed words ["'s", 'and', 'anyon', 'appoin', 'ar', 'assist', 'book', 'bye', 'can', 'cough', 'dat', 'day', 'fev', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hol', 'hour', 'how', 'i', 'is', 'lat', 'me', 'musc', 'my', 'nee', 'of', 'pain', 'pleas', 'see', 'stomach', 'thank', 'that', 'ther', 'tim', 'today', 'u', 'what', 'with', 'yo', 'you']
23 documents
7 classes ['book', 'goodbye', 'greeting', 'help', 'symptoms', 'thanks', 'today']


In [8]:
print("create data for the model...")
training=[]
output=[]
print("create empty list for output...")
output_empty=[0] * len(classes)
print("create training set,bag of words for model")
for doc in documents:
    #initialize bag of words
    bag=[]
    #list of tokenize words for pattern
    pattern_words=doc[0]
    #stem each word
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
    for wi in words:
        bag.append(1) if wi in pattern_words else bag.append(0)
    # output 0 for each tag and 1 for current tag 
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])

create data for the model...
create empty list for output...
create training set,bag of words for model


In [9]:
print("shuffling randomly and converting to numpy array for faster processing..")
random.shuffle(training)
training=np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])
print("building neural network for chatbot to contextual... ")
print("resetting graph data ")
tf.compat.v1.reset_default_graph()

shuffling randomly and converting to numpy array for faster processing..
building neural network for chatbot to contextual... 
resetting graph data 


In [10]:
net=tflearn.input_data(shape=[None,len(train_x[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,len(train_y[0]), activation='softmax')
net=tflearn.regression(net)
print('training..')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
training..


In [11]:
model=tflearn.DNN(net,tensorboard_dir="tflearn_logs")

In [12]:
print("training the model.....")
model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)
print("saving the model...")
model.save("model.tflearn")


Training Step: 2999  | total loss: 0.49723 | time: 0.023s
| Adam | epoch: 1000 | loss: 0.49723 - acc: 0.9449 -- iter: 16/23
Training Step: 3000  | total loss: 0.45614 | time: 0.036s
| Adam | epoch: 1000 | loss: 0.45614 - acc: 0.9504 -- iter: 23/23
--
saving the model...
INFO:tensorflow:C:\Users\HP\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [13]:
print("pickel also saved.......")
pickle.dump({"classes":classes,"train_x":train_x,"train_y":train_y,"words":words},open("training_data",'wb'))

pickel also saved.......


In [14]:
print("pickle is loaded..")
data=pickle.load(open('training_data','rb'))
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

with open('intents.json') as js_data:
    intents=json.load(js_data)
    
print('loading the model')   
#load saved model
model.load('./model.tflearn')

pickle is loaded..
loading the model
INFO:tensorflow:Restoring parameters from C:\Users\HP\model.tflearn


In [19]:
def clean_up_sentence(sentence):
    #it tokenize or break constituents of sentence
    sentence_words=nltk.word_tokenize(sentence)
    #stemming=finding root of given word For eg: giving:stem-word=giv likewise
    sentence_words=[stemmer.stem(word.lower())for word in sentence_words]
    
    #Return  the array of bag of words:True and False or 0 and 1 for each word present in the sentence:
    def bow(sentence,words,show_details=False):
        sentence_words=clean_up_sentence(sentence)
        bag=[0]*len(words)
        for s in sentence_words:
            for i,w in enumerate(words):
                if w==s:
                    bag[i]=1
                if show_details:
                    print(" found in bag: %s"%w)
    return(np.array(bag))

Error_Threshold=0.25
print("Error_Thresold=0.25")

def classify(sentence):
    #prediction of to get possibility or probability from the model:
    results=model.predict([bow(sentence,words)])[0]
    #exclude results which are below threshhold
    results=[[i,r] for i,r in enumerate(results)if r>Error_Threshold]
    #Sorting is done as higher confidence answer comes first
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    return return_list

def response(sentence,user_ID='123',show_details=False):
    results=classify(sentence)
    #That means if classification is done then find matching tag
    if results:
    #long loop
       while results:
            for i in intents['intents']:
            #tag matching
              if i['tag']==results[0][0]:
            #random response from high probability order
                 return  print(random.choice(i['responses']))
            results.pop(0)

Error_Thresold=0.25


In [ ]:
while True:
    input_data=input("you-")
    answer=response(input_data)
    answer